In [ ]:
! wget https://github.com/johntdavies/james/raw/main/data/files.tar

In [ ]:
! tar zxf files.tar

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import glob
import os

In [3]:
folders = ['cat_1','cat_2','cat_3','cat_4']



In [5]:
class_x = []
class_y = []

for i in range(0,4):
    a_folder = folders[i]

    pattern = os.path.join(f'train/{a_folder}', '*.png')

    files = glob.glob(pattern)

    train_x = []
    for file in files:
        img = imread(file)
        #img = np.mean(img[:, :, :3], axis=-1)
        # Normalize the image data
        train_x.append(img)
    train_x = np.array(train_x)
    class_x.append(train_x)

    train_y = (i) * np.ones(len(train_x))
    class_y.append(train_y)


final_x = np.concatenate((class_x[0],class_x[1],class_x[2],class_x[3]))#,class_x[4],class_x[5],class_x[6],class_x[7],class_x[8]))
final_y = np.concatenate((class_y[0],class_y[1],class_y[2],class_y[3]))#,class_y[4],class_y[5],class_y[6],class_y[7],class_y[8]))

print(final_x.shape)
print(final_y.shape)



def shuffle(X,Y):
    assert len(X) == len(Y)
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    print(indices)
    # Use the shuffled indices to reorder both X and Y consistently
    X_shuffled = X[indices]
    Y_shuffled = Y[indices]
    print(Y_shuffled)

    return X_shuffled,Y_shuffled


X_train,y_train = shuffle(final_x,final_y)

(60000, 77, 77, 4)
(60000,)
[53988 44335 13852 ... 32911 59229 22661]
[3. 2. 0. ... 2. 3. 1.]


In [6]:
class_x = []
class_y = []

folders2 = []
for i in range(0,4):
    a_folder = folders[i]

    pattern = os.path.join(f'test/{a_folder}', '*.png')

    files = glob.glob(pattern)

    train_x = []
    for file in files:
        img = imread(file)
        #img = np.mean(img[:, :, :3], axis=-1)
        # Normalize the image data
        train_x.append(img)
    train_x = np.array(train_x)
    class_x.append(train_x)

    train_y = (i) * np.ones(len(train_x))
    class_y.append(train_y)


final_x = np.concatenate((class_x[0],class_x[1],class_x[2],class_x[3]))#,class_x[4],class_x[5],class_x[6],class_x[7],class_x[8]))
final_y = np.concatenate((class_y[0],class_y[1],class_y[2],class_y[3]))#,class_y[4],class_y[5],class_y[6],class_y[7],class_y[8]))

print(final_x.shape)
print(final_y.shape)



def shuffle(X,Y):
    assert len(X) == len(Y)
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    print(indices)
    # Use the shuffled indices to reorder both X and Y consistently
    X_shuffled = X[indices]
    Y_shuffled = Y[indices]
    print(Y_shuffled)

    return X_shuffled,Y_shuffled

X_test,y_test = shuffle(final_x,final_y)

(14119, 77, 77, 4)
(14119,)
[1702 2411 2051 ... 5988 6729 8754]
[0. 0. 0. ... 1. 1. 2.]


In [9]:
from tensorflow.keras.utils import to_categorical
y_train_proper = to_categorical(y_train)
y_test_proper = to_categorical(y_test)

In [10]:
X_train_proper = X_train.reshape(60000,77,77,4)

In [11]:
X_test_proper = X_test.reshape(14119,77,77,4)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization

In [13]:
model = Sequential()

# Second convolution layer
model.add(Conv2D(filters=32, kernel_size=(2,2), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

# Third convolution layer
model.add(Conv2D(filters=64, kernel_size=(4,4), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

# Fourth convolution layer
model.add(Conv2D(filters=32, kernel_size=(4,4), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

# Flattening the convolutions
model.add(Flatten())

# First Dense layer
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))

# Second Dense layer
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))

# Output layer
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=10)

In [15]:
model.fit(X_train_proper,y_train_proper,epochs=30,validation_data=(X_test_proper,y_test_proper),batch_size=128,callbacks=early_stop)
model.save('PredictiveModel1.h5')

Epoch 1/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 114s 240ms/step - accuracy: 0.2818 - loss: 1.4147 - val_accuracy: 0.2629 - val_loss: 1.4265
Epoch 2/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 114s 244ms/step - accuracy: 0.3079 - loss: 1.3623 - val_accuracy: 0.2800 - val_loss: 1.3889
Epoch 3/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 116s 247ms/step - accuracy: 0.3324 - loss: 1.3326 - val_accuracy: 0.2898 - val_loss: 1.4417
Epoch 4/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 113s 240ms/step - accuracy: 0.3617 - loss: 1.2949 - val_accuracy: 0.2857 - val_loss: 1.4789
Epoch 5/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 113s 240ms/step - accuracy: 0.4298 - loss: 1.2083 - val_accuracy: 0.2791 - val_loss: 1.6661
Epoch 6/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 113s 241ms/step - accuracy: 0.5154 - loss: 1.0862 - val_accuracy: 0.2675 - val_loss: 1.6966
Epoch 7/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 114s 243ms/step - accuracy: 0.5939 - loss: 0.9406 - val_accuracy: 0.2847 - val_loss: 2.2344
Epoch 8/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 113s 242ms/step - accuracy: 0.6634 -

In [ ]:
import pandas as pd
metrics = pd.DataFrame(model.history.history)
metrics[['loss','val_loss']].plot()

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
predictions = model.predict(X_test_proper)
predictions = np.argmax(predictions, axis=1)
confusion_matrix(y_test,predictions)

In [ ]:
print(classification_report(y_test,predictions))